Implementation of Shallow network
* First separate the frames from video for both positive and negative examples
* Now use VGG16 network to get feature vector for each time frame
* Now using features for time, we will flatten it and apply a dense layer to get output of 2 for possitive or negative.
* Insted of dense layer, LSTM layer can also be used

In [ ]:
import cv2
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense

Converting videos to frames

In [ ]:
path = './data/video.mp4'
vc = cv2.VideoCapture(path)
rval = True
count = 0
name_no = 0
while rval:
    rval, frame = vc.read()
    count += 1
    if count >= 0 & count < 500:
        if (count % 3 == 0):
            name_no += 1
            # frame = frame[100:350, :]
            cv2.imwrite('image/' + str(name_no) + '.png', frame)
vc.release()

In [ ]:
train_dir = './data/train'
validation_dir = './data/validation'
img_width, img_height = 224, 224

In [ ]:
model = VGG16(weights=None, include_top=False, input_shape=(None, 224, 224))
top_model = Sequential()
for layer in model.layers:
    top_model.add(layer)

In [ ]:
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
# top_model.add(LSTM(256, dropout_U = 0.2, dropout_W = 0.2))
top_model.add(Dropout(0.5))
top_model.add(Dense(2, activation='softmax', name='predictions'))

top_model.compile(loss=loss_function,
              optimizer=optimizers.SGD(lr=learning_rate, momentum=momentum, 
                          decay=1e-6, nesterov=True),metrics=[metrics])

train_datagen = ImageDataGenerator(rescale=1. / 255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_height, img_width),
                                                    batch_size=batch_size,class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(validation_data_dir,target_size=(img_height, img_width),
                                                        batch_size=batch_size,class_mode='categorical')

history = top_model.fit_generator(train_generator, samples_per_epoch=nb_train_samples, epochs=epochs,
                                validation_data=validation_generator, validation_steps=nb_validation_samples/batch_size,
                                  verbose=1, callbacks=[time_callback])

top_model.save(path_to_save_model)